<a href="https://colab.research.google.com/github/paulgureghian/Google_Colab_Notebooks/blob/master/Transfer_learning_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Created by Paul A. Gureghian on 12/3/18.**

**This notebook contains a blueprint for implementing transfer learning,**

**in PyTorch using pre-trained networks from TorchVision's 'ImageNet' dataset.**

In [0]:
### Install PyTorch and CUDA
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [2]:
### Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
### Import packages 
import PIL
import time
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict
from torchvision import datasets, transforms, models

In [0]:
### Load images
data_dir = "/content/drive/My Drive/Cat_Dog_data/Cat_Dog_data"

### Define transforms for the training and testing data 
train_transforms = transforms.Compose([transforms.RandomRotation(30),   
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(), 
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])  

test_transforms = transforms.Compose([transforms.Resize(255),   
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])  
                                
### Build the dataloaders
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128)

In [5]:
### Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load the 'DenseNet' model
model = models.densenet121(pretrained=True) 

### Freeze parameters in order not to backprop through them 
### Turn off gradients  
for param in model.parameters():
    param.requires_grad = False
    
### Define a new classifier    
model.classifier = nn.Sequential(OrderedDict([
                           ('fc1', nn.Linear(1024, 500)),
                           ('relu', nn.ReLU()),
                           ('fc2', nn.Linear(500, 2)),
                           ('output', nn.LogSoftmax(dim=1)) 
                           ]))

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001) 

model.to(device); 

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [6]:
### Train the model  
### Set params
epochs = 1
steps = 0
print_every = 5
running_loss = 0    

### Training loop
for epoch in range(epochs):          
    for inputs, labels in trainloader:
        steps += 1    
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        ### Testing loop         
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    ### Calculate the model accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
          
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test Accuracy: {accuracy/len(testloader):.3f}")
        
            running_loss = 0
            model.train()       

Epoch 1/1.. Train loss: 0.490.. Test loss: 0.206.. Test Accuracy: 0.939
Epoch 1/1.. Train loss: 0.258.. Test loss: 0.142.. Test Accuracy: 0.941
Epoch 1/1.. Train loss: 0.221.. Test loss: 0.103.. Test Accuracy: 0.961
Epoch 1/1.. Train loss: 0.177.. Test loss: 0.121.. Test Accuracy: 0.951
Epoch 1/1.. Train loss: 0.207.. Test loss: 0.109.. Test Accuracy: 0.958
Epoch 1/1.. Train loss: 0.166.. Test loss: 0.062.. Test Accuracy: 0.978
Epoch 1/1.. Train loss: 0.160.. Test loss: 0.058.. Test Accuracy: 0.978
Epoch 1/1.. Train loss: 0.184.. Test loss: 0.049.. Test Accuracy: 0.982


KeyboardInterrupt: ignored